<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:25%"><img src='https://www.np.edu.sg/PublishingImages/Pages/default/odp/ICT.jpg' style="width: 250px; height: 125px; "></th>
        <th style="text-align:center;"><h1>Deep Learning</h1><h2>Assignment 2 - Character Generator Model (Problem 2)</h2><h3>AY2020/21 Semester</h3></th>
    </tr>
</table>

Objective: TODO

TODO: Use perplexity as a validation metric

TODO: Use pretrained Transformer-XL  model from https://github.com/kimiyoung/transformer-xl

In [1]:
DATA_DIR = "../data"

In [18]:
%load_ext lab_black
# Import the Required Packages
import os
import mlflow
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt

from git import Repo
from unidecode import unidecode
from minio import Minio
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Configure access to MLFlow by setting the following environment variables:
- `MLFLOW_TRACKING_URI` - URL to the MLFlow Tracking server.
- `MLFLOW_S3_ENDPOINT_URL` - URL to the MLFlow S3 Backend Store
- `MLFLOW_EXPERIMENT` - Optional. The name of the MLFlow experiment to log to.
- `MINIO_HOST` - End to the Minio S3 Store.
- `AWS_ACCESS_KEY_ID` - MLFlow S3 backend store Avectorsccess Key ID.
- `AWS_SECRET_ACCESS_KEY` - MLFlow S3 backend store secret access key.

`TF_FORCE_GPU_ALLOW_GROWTH` -  Force Tensorflow to allocate GPU memory dynamically
instead of of all at once as a workaround for this
[cuDNN failed to initialize issue](https://github.com/tensorflow/tensorflow/issues/24828).

In [3]:
%env TF_FORCE_GPU_ALLOW_GROWTH=true

env: TF_FORCE_GPU_ALLOW_GROWTH=true


Start MLFlow run with the name of the commit as fthe run name.

In [4]:
mlflow.set_experiment(os.environ.get("MLFLOW_EXPERIMENT", "staging"))

In [5]:
repo = Repo(search_parent_directories=True)
mlflow.start_run(run_name=repo.head.commit.message)

<ActiveRun: >

Setup `minio` client.

In [6]:
minio = Minio(
    endpoint=os.environ["MINIO_HOST"],
    access_key=os.environ["AWS_ACCESS_KEY_ID"],
    secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    secure=False,
)

## Step 1 – Data Loading and Processing

### 1.1 Data Loading

Load the `THE ADVENTURES OF SHERLOCK HOLMES` text:

In [8]:
with open(f"{DATA_DIR}/holmes.txt") as f:
    holmes_text = f.read()

### 1.2 EDA

Take a peek at the first 5 lines:
- a unicode character `\ufeff` has to be replaced.

In [10]:
holmes_text.splitlines()[:5]

['\ufeffTHE ADVENTURES OF SHERLOCK HOLMES by SIR ARTHUR CONAN DOYLE',
 '',
 '   I. A Scandal in Bohemia',
 '  II. The Red-headed League',
 ' III. A Case of Identity']

### 1.3 Data Preprocessing

Replace unicode characters with ASCII ones:

In [14]:
clean_holmes_text = unidecode(holmes_text)

In [16]:
clean_holmes_text.splitlines()[:5]

['THE ADVENTURES OF SHERLOCK HOLMES by SIR ARTHUR CONAN DOYLE',
 '',
 '   I. A Scandal in Bohemia',
 '  II. The Red-headed League',
 ' III. A Case of Identity']

To train a character level language model that predicts the next character, text has to processed into:
- context - List of preceeding characters as the context the model base its prediction on.
- character - The target prediction character given the preceeding context that model is trying to predict.

In [19]:
process_params = {
    "context_len": 10,
}
mlflow.log_params(process_params)

Use a Keras `Sequence` process data at model runtime:
- Data preprocessing is computationally inexpensive, so the performance overhead should be minimal.
- Since there is alot of overlap between context characters, preprocessing the data now will result significant redudant RAM  usage.

In [40]:
class TextContext(Sequence):
    def __init__(self, text, batch_size, process_params):
        """
        Create a text context data sequence useful for character level language models.

        Args:
            text: Text to generate from.
            batch_size: No. of batchs to generate.
            process_params: Data processing parameters
        """
        self.text = text
        self.batch_size = batch_size
        self.context_len = process_params["context_len"]

    def __len__(self):
        # no. of examples: (context, char) pairs
        context_char_len = self.context_len + 1
        n_examples = (len(self.text) - context_char_len) + 1
        return n_examples // self.batch_size

    def __getitem__(self, batch_idx):
        """Process text context data for the given batch at index batch_idx"""
        contexts = []
        target_chars = []

        for i_example in range(self.batch_size):
            # calculate position of where the context vector start and ends
            context_start = batch_idx * self.batch_size + i_example
            context_end = context_start + self.context_len
            # extract context vector
            context = self.text[context_start:context_end]
            contexts.append(context)

            # use character right after context vector as target predict character
            target_char = self.text[context_end]
            target_chars.append(target_char)

        return np.asarray(contexts), np.asarray(target_chars)

In [44]:
holmes_ctx = TextContext(
    text=clean_holmes_text, batch_size=32, process_params=process_params
)
%timeit holmes_ctx.__getitem__(0)

14.7 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Step 2 – Develop Character Generator Model

### Model #1

In [ ]:

#model.save('chgen_model_1.h5')

### Model #2

In [ ]:

#model.save('chgen_model_2.h5')

### Recommend the Best Model

In [ ]:
# Save the Best Model
#model.save('chgen_model_best.h5')

## Step 3 – Use the Best Model to generate the characters / sentences

In [ ]:
#model.load_weights('chgen_model_best.h5')

In [ ]:
# takes the user input
#text_input = np.array([input()])

In [ ]:
# encode the user input



In [ ]:
# Use the Best Model to generate 400 characters

